In [55]:
import pandas as pd
import yfinance as yf
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import FactorAnalysis
import plotly.subplots as sp
import statsmodels.api as sm

Los datos de los 5 factores Fama-French se tomarán para los años 2020 a 2024, es decir del 2020-01-01 al 2024-12-31.

In [56]:
start_date = "2015-01-01"
end_date = "2025-11-04"

In [57]:
#Cargando los 10 ETF Sectoriales de S&P500
#XLC
etfs_name = ['XLF', 'XLK', 'XLV', 'XLE', 'XLI', 'XLY', 'XLP', 'XLU', 'XLRE', 'XLB']

efts_data = []

for name in etfs_name:
    
    data = yf.download(name, start=start_date, end=end_date)
    
    if data.empty:
            print(f"No hay datos para {name}")
    else:
        data.index = data.index.normalize()

        #Modificando para obtener la data mensual
        monthly_data = data[['Close']].resample('M').last()
        monthly_data[name] = monthly_data['Close'].pct_change()
        efts_data.append(monthly_data[name])

C:\Users\Dania\AppData\Local\Temp\ipykernel_32908\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\Dania\AppData\Local\Temp\ipykernel_32908\530410504.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Dania\AppData\Local\Temp\ipykernel_32908\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\Dania\AppData\Local\Temp\ipykernel_32908\530410504.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Dania\AppData\Local\Temp\ipykernel_32908\530410504.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
C:\Users\Dania\AppD

In [58]:
#Uniendo todos los datasets en uno solo
asset_returns_df = pd.concat(efts_data, axis=1)
asset_returns_df.index = pd.to_datetime(asset_returns_df.index)

#Modificando el date a formato yyyymm
asset_returns_df.index = asset_returns_df.index.strftime('%Y%m')
asset_returns_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201502,0.058235,0.079949,0.042876,0.045930,0.053510,0.085441,0.041441,-0.063949,NaN,0.079681
201503,-0.006160,-0.034355,0.006406,-0.011553,-0.025461,-0.004830,-0.019426,-0.009953,NaN,-0.049049
201504,0.000829,0.027510,-0.010897,0.065739,-0.002510,-0.000531,-0.007591,-0.004727,NaN,0.033621
201505,0.019478,0.018553,0.045043,-0.051887,0.003236,0.013146,0.008683,0.006331,NaN,0.003768
...,...,...,...,...,...,...,...,...,...,...
202507,0.000000,0.037555,-0.032347,0.028299,0.030436,0.018865,-0.014697,0.049106,-0.000241,-0.000911
202508,0.030934,-0.001104,0.053669,0.036464,0.000000,0.046561,0.012534,-0.015758,0.021734,0.051864
202509,0.001062,0.075337,0.017292,-0.003182,0.018796,0.035914,-0.023177,0.041236,0.003346,-0.024217


In [59]:
# Cargando los 5 Fama-French factores

raw_data_ff = pd.read_csv('MonthlyFF4F.csv', skiprows=3)

#Pequeña limpieza de nombre de columnas
ff_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
fama_french_df = raw_data_ff.rename(columns={raw_data_ff.columns[0]: 'Date'})
fama_french_df.set_index('Date', inplace=True)
fama_french_df[ff_cols] = fama_french_df[ff_cols] / 100
fama_french_df.index = fama_french_df.index.astype(str)
fama_french_df

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
196307,-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027
196308,0.0508,-0.0080,0.0170,0.0040,-0.0038,0.0025
196309,-0.0157,-0.0043,0.0000,-0.0078,0.0015,0.0027
196310,0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029
196311,-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027
...,...,...,...,...,...,...
202503,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038


In [60]:
#Se hace un inner join para mentener informacion solo de las fechas dadas
final_df = asset_returns_df.join(fama_french_df,on='Date')

final_df = final_df.dropna()

final_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,
201511,0.019934,0.007101,-0.003224,-0.000147,0.008661,-0.002347,-0.009222,-0.021257,-0.006462,0.009939,0.0057,0.0330,-0.0043,-0.0265,-0.0106,0.0000
201512,-0.023644,-0.020811,0.017229,-0.105170,-0.025724,-0.028431,0.029177,0.021192,0.025327,-0.043401,-0.0215,-0.0295,-0.0257,0.0034,0.0012,0.0001
201601,-0.088544,-0.037124,-0.077190,-0.034980,-0.056970,-0.051817,0.005347,0.049446,-0.063943,-0.107093,-0.0574,-0.0344,0.0208,0.0278,0.0305,0.0001
201602,-0.029006,-0.006547,-0.003611,-0.028002,0.042608,0.004453,0.003152,0.019375,0.004796,0.078153,-0.0007,0.0087,-0.0061,0.0331,0.0194,0.0002
201603,0.071592,0.088218,0.027075,0.101659,0.069735,0.066974,0.047383,0.079764,0.106971,0.075731,0.0695,0.0100,0.0122,0.0068,0.0001,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202503,-0.041952,-0.082860,-0.015813,0.034932,-0.035630,-0.083147,-0.011764,0.002394,-0.023920,-0.026703,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.021080,0.016902,-0.037943,-0.138577,0.001144,-0.001013,0.001959,0.000634,-0.013142,-0.024308,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.045119,0.099729,-0.055741,0.012795,0.088401,0.083849,0.012220,0.038276,0.010412,0.029205,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038


In [61]:
#Calculando los excedentes
excess_returns_df = final_df[etfs_name].subtract(final_df['RF'], axis=0)

excess_returns_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201511,0.019934,0.007101,-0.003224,-0.000147,0.008661,-0.002347,-0.009222,-0.021257,-0.006462,0.009939
201512,-0.023744,-0.020911,0.017129,-0.105270,-0.025824,-0.028531,0.029077,0.021092,0.025227,-0.043501
201601,-0.088644,-0.037224,-0.077290,-0.035080,-0.057070,-0.051917,0.005247,0.049346,-0.064043,-0.107193
201602,-0.029206,-0.006747,-0.003811,-0.028202,0.042408,0.004253,0.002952,0.019175,0.004596,0.077953
201603,0.071392,0.088018,0.026875,0.101459,0.069535,0.066774,0.047183,0.079564,0.106771,0.075531
...,...,...,...,...,...,...,...,...,...,...
202503,-0.045352,-0.086260,-0.019213,0.031532,-0.039030,-0.086547,-0.015164,-0.001006,-0.027320,-0.030103
202504,-0.024580,0.013402,-0.041443,-0.142077,-0.002356,-0.004513,-0.001541,-0.002866,-0.016642,-0.027808
202505,0.041319,0.095929,-0.059541,0.008995,0.084601,0.080049,0.008420,0.034476,0.006612,0.025405


In [62]:
fig = px.scatter_matrix(excess_returns_df, width=1200, height=900, title='<b>Scatter matrix para los rendimientos en exceso de los EFTs</b>')

fig.update_layout(title_x = 0.5)

fig.show()

In [63]:
#Matriz de correlacion

correlation_matrix = excess_returns_df.corr()

# heatmap con Plotly
fig = px.imshow(
    correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    aspect="auto",
    text_auto=".2f"
)

fig.update_layout(
    title='<b>Matriz de Correlación para los rendimientos en exceso de los EFTs </b>',
    width=800,
    height=800,
    xaxis_title="Variables",
    yaxis_title="Variables",
    title_x =0.5
)

fig.show()

Hay diversas variables que tienen una correlación bastante alta $(>0.8)$, se puede ver como los siguientes grupos de variables mantienen una correlación alta entre sí:

- Grupo 1: XLB, XLI, XLF

- Grupo 2: XLK, XLY

In [64]:
#Estandarizacion
scaler = StandardScaler()

excess_scaled = scaler.fit_transform(excess_returns_df)
excess_scaled = pd.DataFrame(excess_scaled, columns=excess_returns_df.columns, index=excess_returns_df.index)

excess_scaled

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201511,0.173942,-0.166718,-0.218126,-0.090161,-0.028222,-0.204278,-0.400486,-0.661960,-0.229449,0.050262
201512,-0.594582,-0.671762,0.276084,-1.306561,-0.667420,-0.651221,0.658535,0.302949,0.405643,-0.932490
201601,-1.736528,-0.965864,-2.016578,-0.494378,-1.246587,-1.050398,-0.000395,0.946706,-1.383443,-2.103788
201602,-0.690687,-0.416400,-0.232367,-0.414794,0.597316,-0.091620,-0.063852,0.259272,-0.007819,1.301041
201603,1.079377,1.292137,0.512735,1.085552,1.100131,0.975566,1.159204,1.635237,2.039897,1.256503
...,...,...,...,...,...,...,...,...,...,...
202503,-0.974796,-1.849957,-0.606374,0.276409,-0.912187,-1.641511,-0.564798,-0.200544,-0.647461,-0.686098
202504,-0.609297,-0.053118,-1.146135,-1.732466,-0.232413,-0.241250,-0.188087,-0.242919,-0.433467,-0.643896
202505,0.550230,1.434767,-1.585607,0.015625,1.379386,1.202150,0.087355,0.607915,0.032572,0.334686


In [65]:
#Aplicando PCA
pca = PCA()

pca.fit_transform(excess_scaled)

array([[-0.50024464, -0.62704181,  0.0071915 , ..., -0.00770918,
        -0.16819204, -0.03774211],
       [-1.03876102,  1.71270479,  0.00923518, ..., -0.34750558,
         0.21390304,  0.07230206],
       [-3.42333772,  1.46343537, -0.35599492, ..., -0.03566832,
         0.56798978,  0.23404498],
       ...,
       [ 1.33301208, -0.48134903,  1.34648945, ...,  0.10697078,
        -0.04058563,  0.40685842],
       [ 0.71161892, -0.92253326,  0.67670761, ..., -0.51401117,
        -0.61100206,  0.12239777],
       [-0.22134681,  0.07554058,  0.289347  , ..., -0.07263127,
        -0.07702099,  0.36620693]], shape=(117, 10))

In [66]:
print("Eigenvalores (varianzas explicadas):", pca.explained_variance_)
print("Eigenvectores (componentes principales):\n", pca.components_)
print("Varianza explicada (%):", pca.explained_variance_ratio_ * 100)

Eigenvalores (varianzas explicadas): [6.5579541  1.12363102 0.76858454 0.4664129  0.30548975 0.27397389
 0.2246909  0.16369801 0.12171433 0.08005746]
Eigenvectores (componentes principales):
 [[ 0.3340089   0.30754805  0.31280008  0.25681843  0.3648184   0.32715054
   0.30469048  0.23937129  0.33198428  0.35997812]
 [-0.31430875 -0.1040473   0.07832526 -0.42065097 -0.16131647 -0.16452509
   0.38676944  0.63097601  0.30863298 -0.10599501]
 [-0.17843006  0.58088292 -0.11861264 -0.5485804  -0.01819355  0.4858017
  -0.21227085 -0.14749012  0.09102551 -0.065545  ]
 [-0.07047593 -0.06174885  0.78735431 -0.29872999 -0.05162361 -0.15004972
   0.17563816 -0.4707655  -0.04870651  0.04508487]
 [-0.30040516  0.44934361 -0.04873606  0.39382097 -0.13067956 -0.01007524
   0.56648046 -0.1242528  -0.25866797 -0.36048614]
 [ 0.36120081 -0.02355939 -0.35164717 -0.45218931  0.39653554 -0.18199439
   0.45588865 -0.12311495 -0.34525129  0.09107922]
 [ 0.12980283 -0.26653291 -0.27812666  0.01759827 -0.146121

In [67]:
explained_var = pca.explained_variance_ratio_
cum_var = np.cumsum(explained_var)
components = np.arange(1, len(explained_var)+1)

# Gráfico Plotly
fig_var = go.Figure()

# Varianza explicada por componente
fig_var.add_trace(go.Bar(
    x=components,
    y=explained_var,
    name='Varianza individual',
    marker_color='royalblue'
))

# Varianza acumulada
fig_var.add_trace(go.Scatter(
    x=components,
    y=cum_var,
    name='Varianza acumulada',
    mode='lines+markers',
    line=dict(color='firebrick', width=3),
    marker=dict(size=8)
))

fig_var.update_layout(
    title='Varianza explicada por componente y varianza acumulada',
    xaxis_title='Componente principal',
    yaxis_title='Fracción de varianza',
    yaxis=dict(range=[0,1.05]),
    #legend=dict(x=0.7, y=0.75),
    width=900,
    height=700
)

fig_var.show()

In [68]:
#Calculando los loadings

eigenvectors = pca.components_
eigenvalues = pca.explained_variance_

loadings = eigenvectors * np.sqrt(eigenvalues[:, np.newaxis])

loadings_df =  pd.DataFrame(loadings, columns=excess_scaled.columns, index= [f"PC{i}" for i in range(1, 11)])

loadings_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
PC1,0.855347,0.787585,0.801034,0.657674,0.934245,0.837784,0.780267,0.612994,0.850162,0.921850
PC2,-0.333172,-0.110292,0.083026,-0.445896,-0.170998,-0.174399,0.409981,0.668844,0.327155,-0.112356
PC3,-0.156428,0.509254,-0.103986,-0.480935,-0.015950,0.425897,-0.186096,-0.129303,0.079801,-0.057463
PC4,-0.048131,-0.042171,0.537719,-0.204016,-0.035256,-0.102476,0.119951,-0.321507,-0.033264,0.030790
PC5,-0.166037,0.248357,-0.026937,0.217669,-0.072228,-0.005569,0.313100,-0.068676,-0.142969,-0.199245
PC6,0.189062,-0.012332,-0.184061,-0.236687,0.207557,-0.095260,0.238624,-0.064441,-0.180713,0.047673
PC7,0.061529,-0.126341,-0.131836,0.008342,-0.069264,0.068632,0.137503,-0.223150,0.317163,-0.093218
PC8,-0.241987,-0.117443,-0.066165,0.022384,0.033017,0.118735,0.074519,-0.058483,-0.047865,0.244981
PC9,0.052465,-0.150989,0.059221,-0.012369,-0.003025,0.236407,0.037300,0.056363,-0.114220,-0.137813
PC10,-0.103942,-0.025751,0.015708,0.014755,0.225556,-0.025152,-0.043311,-0.012613,0.048094,-0.110771


La primer componente contiene los loadings de los EFTs altos y positivos, por lo que se mueven en la misma dirección, esto se interpreta como el mercado en general, pues cuando el "mercado" en general sube, todos suben o bajan. Sin embargo hay algunos un poco mas altos como 

- XLI (0.93): Industriales
- XLU (0.92): Finanzas
- XLE (0.65): Energía

La segunda componente muestra otro grupo, ya que separa a los componentes en dos sectores:

- Positivos con XLU (Utilidades), XLP (Consumo Básico), XLRE (Bienes raices)
- Negativos con XLE (Energía), XLF (Finanzas)

La tercera componente tiene menos componentes representativas, sin embargo con PCA se logró encontrar poca información de la interpretación de las variables, ya que a partir del 3er grupo es mucho ruido y complicado de determinar si realmente es un riesgo de mercado o un patrón estadístico únicamente.

In [69]:
#Analisis Factorial
#aplicamos analisis factorial con 3 factores y la rotacion varimax
fa = FactorAnalysis(n_components=3, rotation='varimax')
fa_factors = fa.fit_transform(excess_scaled)

fa_factors_df = pd.DataFrame(fa_factors, columns=['Factor1', 'Factor2', 'Factor3'],
                            index=excess_scaled.index)

# Obtener las cargas factoriales
factor_loadings = fa.components_.T
factor_loadings_df = pd.DataFrame(factor_loadings,
                                 columns=['Factor1', 'Factor2', 'Factor3'],
                                 index=excess_returns_df.columns)

#visualizar loadings factoriales
fig_fa_loadings = px.imshow(factor_loadings_df,
    color_continuous_scale='RdBu_r',
    aspect="auto",
    text_auto=".2f",
    title='Loadings Factoriales Rotados (Varimax)'
)
fig_fa_loadings.show()

# Calcular comunalidades y varianza específica
communalities = np.sum(factor_loadings**2, axis=1)
specific_variances = 1 - communalities
fa_results_df = pd.DataFrame({
    'Comunalidad': communalities,
    'Varianza_Especifica': specific_variances
}, index=excess_returns_df.columns)
fa_results_df

,Comunalidad,Varianza_Especifica
XLF,0.854217,0.145783
XLK,0.745236,0.254764
XLV,0.596171,0.403829
XLE,0.553911,0.446089
XLI,0.913829,0.086171
XLY,0.937704,0.062296
XLP,0.728351,0.271649
XLU,0.661622,0.338378
XLRE,0.801184,0.198816
XLB,0.850223,0.149777


## Comparación: PCA vs Análisis Factorial

Diferencias entre Factores Comunes y Específicos


PCA


**PC1 explica**: 65.0% de la varianza total


**PC2 explica**: 11.1% de la varianza total  


**PC3 explica**: 7.6% de la varianza total

FA


**Comunalidad promedio**: 76.4%


**Varianza específica promedio**: 23.6%

Comparación Directa

Ventajas PCA
- Captura máxima varianza con mínimo número de componentes
- PC1 claramente identifica el riesgo sistemático de mercado
- No requiere supuestos sobre la estructura factorial

Ventajas FA
- Rotación Varimax produce factores más interpretables económicamente
- Separa claramente tres grupos sectoriales:
  - Tecnología/Consumo Discrecional
  - Finanzas/Energía/Industriales  
  - Defensivos (Utilidades/Consumo Básico/Real Estate)
- Proporciona medidas de comunalidad y varianza específica

ETFs Mejor Explicados por Cada Método

PCA (ETFs con mayor carga en PC1)
- XLI: 0.934 (Industriales)
- XLB: 0.922 (Materiales)
- XLF: 0.855 (Finanzas)

FA - (ETFs con mayor comunalidad (mejor explicados por factores comunes))
- XLY: 0.938 (Consumo Discrecional)
- XLI: 0.914 (Industriales)
- XLF: 0.854 (Finanzas)

Recomendación:
- Usar PCA para reducción dimensional y detección de riesgo sistemático
- Usar FA para identificar factores económicos latentes y agrupaciones sectoriales

In [ ]:
# Paso 7: Modelo Fama-French 5 factores
#preparar los datos para regresion
X_ff = final_df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
X_ff = sm.add_constant(X_ff)

results_ff = {}
alphas = {}
betas = {}
r_squared = {}

#regresiones para cada ETF
for etf in etfs_name:
    y = excess_returns_df[etf]
    model = sm.OLS(y, X_ff)
    results = model.fit()
    results_ff[etf] = results
    alphas[etf] = results.params['const']
    betas[etf] = results.params.drop('const')
    r_squared[etf] = results.rsquared

alpha_df = pd.DataFrame.from_dict(alphas, orient='index', columns=['Alpha'])
beta_df = pd.DataFrame.from_dict(betas, orient='index')
r_squared_df = pd.DataFrame.from_dict(r_squared, orient='index', columns=['R²'])

print("Betas de los factores Fama-French por ETF:")
print(beta_df.round(4))

results_summary = pd.concat([alpha_df, r_squared_df], axis=1)
print("\nAlfas y R² del modelo Fama-French:")
print(results_summary.round(4))

print("Significancia estadística de los Alfas:")
for etf in etfs_name:
    alpha_pvalue = results_ff[etf].pvalues['const']
    significance = "***" if alpha_pvalue < 0.01 else "**" if alpha_pvalue < 0.05 else "*" if alpha_pvalue < 0.1 else ""
    
    print(f"{etf}: Alpha = {alphas[etf]:.4f}, p-value = {alpha_pvalue:.4f} {significance}")

Betas de los factores Fama-French por ETF:
      Mkt-RF     SMB     HML     RMW     CMA
XLF   1.0212 -0.1111  0.8170 -0.2822 -0.3658
XLK   1.1232 -0.1984 -0.3146  0.1382 -0.0758
XLV   0.7147 -0.0661 -0.1346  0.1099  0.3841
XLE   1.1622  0.1798  0.9490 -0.2208  0.5484
XLI   1.0379  0.1434  0.2138  0.2289  0.0275
XLY   1.1037  0.1147 -0.1680  0.0555 -0.2665
XLP   0.6038 -0.2264 -0.0590  0.4222  0.4780
XLU   0.5186 -0.2300  0.0093  0.3189  0.2856
XLRE  0.8034  0.1189 -0.0922  0.2775  0.1624
XLB   0.9980  0.1498  0.1745  0.3206  0.1943

Alfas y R² del modelo Fama-French:
       Alpha      R²
XLF   0.0011  0.9131
XLK   0.0034  0.8848
XLV  -0.0022  0.5715
XLE  -0.0019  0.6745
XLI  -0.0008  0.8734
XLY  -0.0021  0.8531
XLP  -0.0028  0.6261
XLU   0.0009  0.2999
XLRE -0.0041  0.5703
XLB  -0.0037  0.7954

Significancia estadística de los Alfas:
XLF: Alpha = 0.0011, p-value = 0.5007 
XLK: Alpha = 0.0034, p-value = 0.0711 *
XLV: Alpha = -0.0022, p-value = 0.4093 
XLE: Alpha = -0.0019, p-value = 0.6

| Factor | Estrategia si Beta es POSITIVO | Estrategia si Beta es NEGATIVO |
|--------|--------------------------------|--------------------------------|
| **Mkt-RF** | Sensible al mercado | Defensivo/Contracíclico |
| **SMB** | Exposición a pequeñas empresas | Exposición a grandes empresas |
| **HML** | Estrategia Value | Estrategia Growth |
| **RMW** | Empresas rentables | Empresas menos rentables |
| **CMA** | Inversión conservadora | Inversión agresiva |

## Análisis de los Resultados Fama-French 5 Factores

Interpretación de los Betas por Sector

XLF (Finanzas) - R²: 91.3%
- Alto Beta HML (0.817): Empresas financieras típicamente tienen alto book-to-market
- Beta Mkt-RF (1.02): Movimiento similar al mercado
- Betas RMW y CMA negativos: Rentabilidad operativa moderada y políticas de inversión menos conservadoras

XLK (Tecnología) - R²: 88.5%
- Beta HML negativo (-0.315): Empresas tech con alto crecimiento esperado
- Alto Beta Mkt-RF (1.12): Alta sensibilidad al mercado
- Alpha significativo (0.0034): Posible sobreperformance no explicada por los factores

XLE (Energía) - R²: 67.4%
- Betas HML (0.949) y CMA (0.548) muy altos: Fuertes características VALUE y políticas de inversión conservadoras
- Beta Mkt-RF alto (1.16): Muy sensible a movimientos del mercado

XLP (Consumo Básico) & XLU (Utilidades)
- Betas Mkt-RF bajos (0.60 y 0.52): Defensivos - menos sensibles al mercado
- Betas RMW y CMA positivos: Empresas rentables con políticas conservadoras
- R² moderados: Otros factores explican parte de su variabilidad


Poder Explicativo (R²)

- Alto R² (>85%): XLF, XLK, XLI, XLY - bien explicados por los factores
- R² Moderado (60-80%): XLE, XLB, XLP
- Bajo R² (<60%): XLV, XLRE, XLU - otros factores importantes no capturados

El asterisco (*) en XLK (p-value = 0.0711) indica:
Significancia estadística igaul o menor al 10%, lo que significa que hay evidencia débil de que XLK genera retornos positivos no explicados por los factores Fama-French.

In [ ]:
# Cuadro comparativo
# PCA
pca_variance = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(len(explained_var))],
    'Varianza_Explicada': explained_var,
    'Varianza_Acumulada': cum_var
})

# Analisis factorial
fa_variance = pd.DataFrame({
    'Metrica': ['Comunalidad_Promedio', 'Varianza_Especifica_Promedio'],
    'Valor': [fa_results_df['Comunalidad'].mean(), fa_results_df['Varianza_Especifica'].mean()]
})

# Resumen Fama-French
ff_summary = pd.DataFrame({
    'ETF': etfs_name,
    'Alpha': [alphas[etf] for etf in etfs_name],
    'R2': [r_squared[etf] for etf in etfs_name],
    'Beta_MKT': [betas[etf]['Mkt-RF'] for etf in etfs_name],
    'Beta_SMB': [betas[etf]['SMB'] for etf in etfs_name],
    'Beta_HML': [betas[etf]['HML'] for etf in etfs_name]
})

print("Cuadro Comparatibo de los 3 metodos")

print("PCA:")
print(f"Primer componente explica: {explained_var[0]*100:.2f}% de la varianza")
print(f"Primeros 3 componentes explican: {cum_var[2]*100:.2f}% de la varianza")
print(f"Se necesitan 3 componentes para explicar 80% de la varianza")
print("--------------------------------")
print("Analisis Factorial (3 Factores):")
print(f"Comunalidad promedio: {fa_results_df['Comunalidad'].mean():.4f}")
print(f"Varianza específica promedio: {fa_results_df['Varianza_Especifica'].mean():.4f}")
print("ETFs con mayor comunalidad (más explicados por factores comunes):")
top_communal = fa_results_df.nlargest(3, 'Comunalidad')
for etf, row in top_communal.iterrows():
    print(f"     {etf}: {row['Comunalidad']:.4f}")
print("--------------------------------")
print("MODELO FAMAMA-FRENCH (5 FACTORES):")
print(f"R² promedio: {r_squared_df.mean().values[0]:.4f}")
print(f"ETFs mejor explicados por el modelo (mayor R²):")
top_r2 = r_squared_df.nlargest(3, 'R²')
for etf, row in top_r2.iterrows():
    print(f"     {etf}: {row['R²']:.4f}")

Cuadro Comparatibo de los 3 metodos
PCA:
Primer componente explica: 65.02% de la varianza
Primeros 3 componentes explican: 83.78% de la varianza
Se necesitan 3 componentes para explicar 80% de la varianza
--------------------------------
Analisis Factorial (3 Factores):
Comunalidad promedio: 0.7642
Varianza específica promedio: 0.2358
ETFs con mayor comunalidad (más explicados por factores comunes):
     XLY: 0.9377
     XLI: 0.9138
     XLF: 0.8542
--------------------------------
MODELO FAMAMA-FRENCH (5 FACTORES):
R² promedio: 0.7062
ETFs mejor explicados por el modelo (mayor R²):
     XLF: 0.9131
     XLK: 0.8848
     XLI: 0.8734


## Conclusiones

¿Qué método ofrece factores más interpretables?

PCA tiene interpretabilidad limitada después del primer componente (riesgo mercado general)

Análisis Factorial con la rotación Varimax nos da una buena interpretabilidad sectorial:
- Crecimiento, Cíclicos, Defensivos

Fama-French ofrece la mejor interpretabilidad con los factores económicos predefinidos:
- Mkt-RF (riesgo mercado), SMB (tamaño), HML (valor/crecimiento), RMW (rentabilidad), CMA (inversión)


¿Cómo se complementan?

Métodos estadísticos (PCA y FA) identifican patrones en los datos:
- Estructura de correlaciones y agrupaciones naturales

Factores empíricos (Fama-French) explican el "porque" económico:
- Cuantifican exposición a riesgos económicos específicos

O en otras palabras PCA/FA muestran "qué" patrones existen, Fama-French explica "por qué" existen y su significado económico.